## कद्दू की कीमत के लिए रैखिक और बहुपद प्रतिगमन - पाठ 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>दासानी मदीपल्ली द्वारा इन्फोग्राफिक</figcaption>


#### परिचय

अब तक आपने कद्दू की कीमत के डेटासेट से एकत्र किए गए नमूना डेटा के साथ प्रतिगमन (Regression) क्या है, इसका पता लगाया है, जिसे हम इस पाठ में उपयोग करेंगे। आपने इसे `ggplot2` का उपयोग करके विज़ुअलाइज़ भी किया है।💪

अब आप मशीन लर्निंग के लिए प्रतिगमन में गहराई से जाने के लिए तैयार हैं। इस पाठ में, आप प्रतिगमन के दो प्रकारों के बारे में अधिक जानेंगे: *मूल रैखिक प्रतिगमन* और *बहुपद प्रतिगमन*, साथ ही इन तकनीकों के पीछे कुछ गणितीय अवधारणाओं को समझेंगे।

> इस पाठ्यक्रम में, हम गणित का न्यूनतम ज्ञान मानते हैं और इसे अन्य क्षेत्रों से आने वाले छात्रों के लिए सुलभ बनाने का प्रयास करते हैं। इसलिए नोट्स, 🧮 गणना, आरेख और अन्य शिक्षण उपकरणों पर ध्यान दें जो समझने में मदद करेंगे।

#### तैयारी

याद दिलाने के लिए, आप इस डेटा को लोड कर रहे हैं ताकि इससे सवाल पूछ सकें।

-   कद्दू खरीदने का सबसे अच्छा समय कब है?

-   मिनिएचर कद्दू के एक केस की कीमत क्या हो सकती है?

-   क्या मुझे उन्हें आधे-बुशल टोकरी में खरीदना चाहिए या 1 1/9 बुशल बॉक्स में? चलिए इस डेटा में और गहराई से जाते हैं।

पिछले पाठ में, आपने एक `tibble` (डेटा फ्रेम का आधुनिक पुनर्कल्पना) बनाया और इसे मूल डेटासेट के हिस्से से भर दिया, बुशल द्वारा कीमत को मानकीकृत किया। हालांकि, ऐसा करने से, आप केवल लगभग 400 डेटा पॉइंट्स और केवल पतझड़ के महीनों के लिए डेटा एकत्र कर पाए। शायद हम डेटा की प्रकृति के बारे में अधिक विवरण प्राप्त कर सकते हैं यदि इसे और अधिक साफ किया जाए? देखते हैं... 🕵️‍♀️

इस कार्य के लिए, हमें निम्नलिखित पैकेजों की आवश्यकता होगी:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) एक [R पैकेजों का संग्रह](https://www.tidyverse.org/packages) है जो डेटा साइंस को तेज़, आसान और मज़ेदार बनाता है!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) फ्रेमवर्क [पैकेजों का संग्रह](https://www.tidymodels.org/packages/) है जो मॉडलिंग और मशीन लर्निंग के लिए उपयोगी है।

-   `janitor`: [janitor पैकेज](https://github.com/sfirke/janitor) गंदे डेटा की जांच और सफाई के लिए सरल उपकरण प्रदान करता है।

-   `corrplot`: [corrplot पैकेज](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) एक विज़ुअल एक्सप्लोरेटरी टूल प्रदान करता है जो कोरिलेशन मैट्रिक्स पर आधारित है और छिपे हुए पैटर्न का पता लगाने के लिए स्वचालित वेरिएबल रीऑर्डरिंग का समर्थन करता है।

आप इन्हें इस प्रकार इंस्टॉल कर सकते हैं:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

नीचे दिया गया स्क्रिप्ट यह जांचता है कि क्या आपके पास इस मॉड्यूल को पूरा करने के लिए आवश्यक पैकेज हैं और यदि वे गायब हैं तो उन्हें आपके लिए इंस्टॉल करता है।


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

हम बाद में इन शानदार पैकेजों को लोड करेंगे और उन्हें हमारे वर्तमान R सत्र में उपलब्ध कराएंगे। (यह केवल उदाहरण के लिए है, `pacman::p_load()` ने पहले ही यह आपके लिए कर दिया है)

## 1. एक रेखीय प्रतिगमन रेखा

जैसा कि आपने पाठ 1 में सीखा, रेखीय प्रतिगमन अभ्यास का उद्देश्य एक *सर्वश्रेष्ठ फिट रेखा* को प्लॉट करना है ताकि:

-   **चर संबंध दिखाएं**। चर के बीच संबंध दिखाएं।

-   **भविष्यवाणी करें**। यह सटीक भविष्यवाणी करें कि नया डेटा बिंदु उस रेखा के संबंध में कहां गिरेगा।

इस प्रकार की रेखा खींचने के लिए, हम एक सांख्यिकीय तकनीक का उपयोग करते हैं जिसे **लीस्ट-स्क्वेयर प्रतिगमन** कहा जाता है। `लीस्ट-स्क्वेयर` का मतलब है कि प्रतिगमन रेखा के चारों ओर के सभी डेटा बिंदुओं को वर्गाकार किया जाता है और फिर जोड़ा जाता है। आदर्श रूप से, वह अंतिम योग जितना छोटा हो सके उतना छोटा होना चाहिए, क्योंकि हम त्रुटियों की कम संख्या चाहते हैं, या `लीस्ट-स्क्वेयर`। इस प्रकार, सर्वश्रेष्ठ फिट रेखा वह रेखा है जो वर्गाकार त्रुटियों के योग के लिए सबसे कम मान देती है - इसलिए इसे *लीस्ट-स्क्वेयर प्रतिगमन* कहा जाता है।

हम ऐसा इसलिए करते हैं क्योंकि हम एक ऐसी रेखा को मॉडल करना चाहते हैं जिसका हमारे सभी डेटा बिंदुओं से संचयी दूरी सबसे कम हो। हम इन शब्दों को जोड़ने से पहले वर्गाकार करते हैं क्योंकि हमें इसकी परिमाण की चिंता है, न कि उसकी दिशा की।

> **🧮 गणित दिखाएं**
>
> इस रेखा, जिसे *सर्वश्रेष्ठ फिट रेखा* कहा जाता है, को [एक समीकरण](https://en.wikipedia.org/wiki/Simple_linear_regression) द्वारा व्यक्त किया जा सकता है:
>
>     Y = a + bX
>
> `X` '`स्पष्टीकरण चर` या `पूर्वानुमानक`' है। `Y` '`निर्भर चर` या `परिणाम`' है। रेखा का ढलान `b` है और `a` y-अवरोधक है, जो उस समय `Y` का मान दर्शाता है जब `X = 0` होता है।
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "slope = $y/x$")
    जेन लूपर द्वारा इन्फोग्राफिक
>
> पहले, ढलान `b` की गणना करें।
>
> दूसरे शब्दों में, और हमारे कद्दू डेटा के मूल प्रश्न का संदर्भ देते हुए: "महीने के अनुसार प्रति बुशल कद्दू की कीमत की भविष्यवाणी करें", `X` कीमत को संदर्भित करेगा और `Y` बिक्री के महीने को।
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.hi.png)
    जेन लूपर द्वारा इन्फोग्राफिक
> 
> `Y` का मान निकालें। अगर आप लगभग \$4 खर्च कर रहे हैं, तो यह अप्रैल होना चाहिए!
>
> रेखा की गणना करने वाला गणित रेखा के ढलान को प्रदर्शित करना चाहिए, जो अवरोधक पर भी निर्भर करता है, या जहां `X = 0` होने पर `Y` स्थित होता है।
>
> आप इन मानों की गणना की विधि [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html) वेबसाइट पर देख सकते हैं। साथ ही [इस लीस्ट-स्क्वेयर कैलकुलेटर](https://www.mathsisfun.com/data/least-squares-calculator.html) पर जाएं और देखें कि कैसे संख्याओं के मान रेखा को प्रभावित करते हैं।

इतना डरावना नहीं, है ना? 🤓

#### सहसंबंध

एक और शब्द जिसे समझना है वह है **सहसंबंध गुणांक** दिए गए X और Y चर के बीच। एक स्कैटरप्लॉट का उपयोग करके, आप इस गुणांक को जल्दी से देख सकते हैं। एक प्लॉट जिसमें डेटा बिंदु एक साफ रेखा में बिखरे होते हैं, उसमें उच्च सहसंबंध होता है, लेकिन एक प्लॉट जिसमें डेटा बिंदु X और Y के बीच हर जगह बिखरे होते हैं, उसमें कम सहसंबंध होता है।

एक अच्छा रेखीय प्रतिगमन मॉडल वह होगा जिसमें लीस्ट-स्क्वेयर प्रतिगमन विधि का उपयोग करके एक रेखा के साथ उच्च (1 के करीब, 0 से दूर) सहसंबंध गुणांक हो।


## **2. डेटा के साथ एक नृत्य: एक डेटा फ्रेम बनाना जो मॉडलिंग के लिए उपयोग किया जाएगा**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>कला कार्य @allison_horst द्वारा</figcaption>


<!--![कला कार्य \@allison_horst द्वारा](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.hi.jpg){width="700"}-->


आवश्यक लाइब्रेरी और डेटा सेट लोड करें। डेटा को एक डेटा फ्रेम में बदलें जिसमें डेटा का एक उपसमूह हो:

-   केवल उन कद्दुओं को प्राप्त करें जिनकी कीमत बुशल के आधार पर दी गई है

-   तारीख को महीने में बदलें

-   कीमत को उच्च और निम्न कीमतों के औसत के रूप में गणना करें

-   कीमत को बुशल मात्रा के अनुसार दर्शाने के लिए बदलें

> हमने इन चरणों को [पिछले पाठ](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb) में कवर किया था।


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

आइए रोमांच की भावना में, [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) का अन्वेषण करें, जो गंदे डेटा की जांच और सफाई के लिए सरल फ़ंक्शन प्रदान करता है। उदाहरण के लिए, आइए हमारे डेटा के कॉलम नामों पर एक नज़र डालें:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 हम बेहतर कर सकते हैं। आइए इन कॉलम नामों को `janitor::clean_names` का उपयोग करके [snake_case](https://en.wikipedia.org/wiki/Snake_case) कन्वेंशन में बदलकर `friendR` बनाएं। इस फ़ंक्शन के बारे में अधिक जानने के लिए: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

बहुत साफ-सुथरा 🧹! अब, पिछले पाठ की तरह `dplyr` का उपयोग करके डेटा के साथ एक नृत्य! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

अच्छा काम!👌 अब आपके पास एक साफ-सुथरा और व्यवस्थित डेटा सेट है, जिस पर आप अपना नया रिग्रेशन मॉडल बना सकते हैं!

क्या एक स्कैटर प्लॉट बनाएंगे?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


एक स्कैटर प्लॉट हमें याद दिलाता है कि हमारे पास केवल अगस्त से दिसंबर तक का मासिक डेटा है। हमें शायद और डेटा की आवश्यकता होगी ताकि हम रेखीय तरीके से निष्कर्ष निकाल सकें।

आइए एक बार फिर अपने मॉडलिंग डेटा पर नज़र डालें:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

अगर हम कद्दू की `price` का अनुमान लगाना चाहें, जो `city` या `package` कॉलम पर आधारित हो, जो कि कैरेक्टर प्रकार के हैं, तो क्या करें? या और भी सरल तरीके से, हम कैसे `package` और `price` के बीच सहसंबंध (जिसके लिए दोनों इनपुट्स का संख्यात्मक होना आवश्यक है) का पता लगा सकते हैं? 🤷🤷

मशीन लर्निंग मॉडल आमतौर पर टेक्स्ट वैल्यू की बजाय संख्यात्मक फीचर्स के साथ बेहतर काम करते हैं, इसलिए आपको सामान्यतः श्रेणीबद्ध फीचर्स को संख्यात्मक प्रतिनिधित्व में बदलने की आवश्यकता होती है।

इसका मतलब है कि हमें अपने प्रेडिक्टर्स को इस तरह से पुनः स्वरूपित करने का तरीका खोजना होगा ताकि मॉडल उनका प्रभावी ढंग से उपयोग कर सके, जिसे `feature engineering` कहा जाता है।


## 3. मॉडलिंग के लिए डेटा को प्रीप्रोसेस करना रेसिपीज़ के साथ 👩‍🍳👨‍🍳

ऐसी गतिविधियाँ जो प्रेडिक्टर मानों को पुनः स्वरूपित करती हैं ताकि मॉडल उन्हें प्रभावी ढंग से उपयोग कर सके, को `फीचर इंजीनियरिंग` कहा गया है।

अलग-अलग मॉडलों की अलग-अलग प्रीप्रोसेसिंग आवश्यकताएँ होती हैं। उदाहरण के लिए, लीस्ट स्क्वेयर को `श्रेणीबद्ध वेरिएबल्स को एनकोड करना` पड़ता है, जैसे कि महीना, प्रकार और शहर का नाम। इसका मतलब है कि `श्रेणीबद्ध मानों` वाले कॉलम को एक या अधिक `संख्यात्मक कॉलम्स` में बदलना, जो मूल कॉलम की जगह लेते हैं।

उदाहरण के लिए, मान लीजिए आपके डेटा में निम्नलिखित श्रेणीबद्ध फीचर शामिल है:

|  शहर   |
|:-------:|
| डेनवर  |
| नैरोबी |
|  टोक्यो |

आप *ऑर्डिनल एनकोडिंग* लागू कर सकते हैं, जो प्रत्येक श्रेणी के लिए एक अद्वितीय पूर्णांक मान को प्रतिस्थापित करता है, इस प्रकार:

| शहर |
|:----:|
|  0   |
|  1   |
|  2   |

और यही हम अपने डेटा के साथ करेंगे!

इस सेक्शन में, हम एक और शानदार Tidymodels पैकेज [recipes](https://tidymodels.github.io/recipes/) का पता लगाएंगे - जिसे आपके डेटा को **मॉडल ट्रेनिंग से पहले** प्रीप्रोसेस करने में मदद करने के लिए डिज़ाइन किया गया है। मूल रूप से, एक रेसिपी एक ऑब्जेक्ट है जो यह परिभाषित करता है कि डेटा सेट पर कौन-कौन से स्टेप्स लागू किए जाने चाहिए ताकि इसे मॉडलिंग के लिए तैयार किया जा सके।

अब, चलिए एक रेसिपी बनाते हैं जो हमारे डेटा को मॉडलिंग के लिए तैयार करती है, प्रेडिक्टर कॉलम्स में सभी ऑब्जर्वेशन्स के लिए एक अद्वितीय पूर्णांक को प्रतिस्थापित करके:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

वाह! 👏 हमने अभी अपना पहला रेसिपी बनाई है जो एक परिणाम (मूल्य) और उसके संबंधित भविष्यवक्ताओं को निर्दिष्ट करती है, और यह सुनिश्चित करती है कि सभी भविष्यवक्ता कॉलम को संख्याओं के सेट में एन्कोड किया जाए 🙌! चलिए इसे जल्दी से समझते हैं:

-   `recipe()` को एक फॉर्मूला के साथ कॉल करना रेसिपी को यह बताता है कि *चर* की भूमिकाएं क्या हैं, और यह `new_pumpkins` डेटा को संदर्भ के रूप में उपयोग करता है। उदाहरण के लिए, `price` कॉलम को `outcome` भूमिका दी गई है, जबकि बाकी कॉलम को `predictor` भूमिका दी गई है।

-   `step_integer(all_predictors(), zero_based = TRUE)` निर्दिष्ट करता है कि सभी भविष्यवक्ताओं को संख्याओं के सेट में परिवर्तित किया जाना चाहिए, जिसमें क्रमांक 0 से शुरू होता है।

हमें यकीन है कि आपके मन में यह विचार आ रहा होगा: "यह तो बहुत शानदार है!! लेकिन अगर मुझे यह सुनिश्चित करना हो कि रेसिपी वही कर रही है जो मैं उनसे उम्मीद कर रहा हूं, तो क्या होगा? 🤔"

यह एक शानदार विचार है! देखिए, एक बार जब आपकी रेसिपी परिभाषित हो जाती है, तो आप डेटा को वास्तव में प्रीप्रोसेस करने के लिए आवश्यक पैरामीटर का अनुमान लगा सकते हैं, और फिर प्रीप्रोसेस किए गए डेटा को निकाल सकते हैं। आमतौर पर आपको यह करने की आवश्यकता नहीं होती जब आप Tidymodels का उपयोग करते हैं (हम इसे सामान्य प्रक्रिया में जल्द ही देखेंगे -> `workflows`), लेकिन यह तब उपयोगी हो सकता है जब आप यह सुनिश्चित करना चाहते हैं कि रेसिपी वही कर रही है जो आप उनसे उम्मीद कर रहे हैं।

इसके लिए, आपको दो और क्रियाओं की आवश्यकता होगी: `prep()` और `bake()`। और हमेशा की तरह, हमारे छोटे R दोस्त [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) आपको इसे बेहतर तरीके से समझने में मदद करते हैं!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Artwork by @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): प्रशिक्षण सेट से आवश्यक पैरामीटर का अनुमान लगाता है, जिसे बाद में अन्य डेटा सेट पर लागू किया जा सकता है। उदाहरण के लिए, किसी दिए गए प्रेडिक्टर कॉलम के लिए, कौन सा ऑब्ज़र्वेशन इंटीजर 0, 1, 2 आदि के रूप में असाइन किया जाएगा।

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): एक तैयार की गई रेसिपी लेता है और किसी भी डेटा सेट पर ऑपरेशन्स लागू करता है।

अब, चलिए अपनी रेसिपी को तैयार करते हैं और लागू करते हैं ताकि वास्तव में पुष्टि हो सके कि बैकग्राउंड में प्रेडिक्टर कॉलम पहले एन्कोड किए जाएंगे, उसके बाद मॉडल फिट किया जाएगा।


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

वाह! 🥳 प्रोसेस किए गए डेटा `baked_pumpkins` के सभी प्रेडिक्टर्स को एन्कोड कर दिया गया है, जिससे यह पुष्टि होती है कि हमारे द्वारा परिभाषित प्रीप्रोसेसिंग स्टेप्स, जो एक रेसिपी के रूप में हैं, अपेक्षित रूप से काम करेंगे। यह आपके लिए पढ़ना थोड़ा मुश्किल बना सकता है, लेकिन Tidymodels के लिए इसे समझना बहुत आसान हो जाता है! कुछ समय निकालें और पता करें कि कौन सा ऑब्ज़र्वेशन संबंधित पूर्णांक में मैप किया गया है।

यह उल्लेख करना भी महत्वपूर्ण है कि `baked_pumpkins` एक डेटा फ्रेम है, जिस पर हम गणनाएँ कर सकते हैं।

उदाहरण के लिए, चलिए आपके डेटा के दो बिंदुओं के बीच एक अच्छा सहसंबंध खोजने की कोशिश करते हैं ताकि संभावित रूप से एक अच्छा प्रेडिक्टिव मॉडल बनाया जा सके। इसके लिए हम `cor()` फ़ंक्शन का उपयोग करेंगे। इस फ़ंक्शन के बारे में अधिक जानने के लिए `?cor()` टाइप करें।


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


जैसा कि पता चलता है, शहर और कीमत के बीच केवल कमजोर संबंध है। हालांकि, पैकेज और उसकी कीमत के बीच थोड़ा बेहतर संबंध है। यह समझ में आता है, है ना? आमतौर पर, जितना बड़ा उत्पाद बॉक्स होता है, कीमत उतनी ही अधिक होती है।

जब हम इस पर हैं, तो चलिए `corrplot` पैकेज का उपयोग करके सभी कॉलम का एक सहसंबंध मैट्रिक्स भी विज़ुअलाइज़ करने की कोशिश करते हैं।


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 बहुत बेहतर।

अब इस डेटा से पूछने के लिए एक अच्छा सवाल होगा: '`किसी दिए गए कद्दू पैकेज की कीमत क्या हो सकती है?`' चलिए इसे शुरू करते हैं!

> नोट: जब आप **`bake()`** करते हैं प्रीप्ड रेसिपी **`pumpkins_prep`** के साथ **`new_data = NULL`**, तो आप प्रोसेस्ड (यानी एनकोडेड) ट्रेनिंग डेटा निकालते हैं। अगर आपके पास कोई दूसरा डेटा सेट है, जैसे कि टेस्ट सेट, और आप देखना चाहते हैं कि रेसिपी उसे कैसे प्री-प्रोसेस करेगी, तो आप बस **`pumpkins_prep`** को **`new_data = test_set`** के साथ बेक करेंगे।

## 4. एक लीनियर रिग्रेशन मॉडल बनाएं

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>दासानी मदीपल्ली द्वारा इन्फोग्राफिक</figcaption>


<!--![दासानी मदीपल्ली द्वारा इन्फोग्राफिक](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.hi.png){width="800"}-->


अब जब हमने एक रेसिपी बनाई है और यह सुनिश्चित कर लिया है कि डेटा को सही तरीके से प्री-प्रोसेस किया जाएगा, तो अब हम एक रिग्रेशन मॉडल बनाएंगे ताकि इस सवाल का जवाब दिया जा सके: `किसी दिए गए कद्दू पैकेज की कीमत क्या हो सकती है?`

#### प्रशिक्षण सेट का उपयोग करके एक लीनियर रिग्रेशन मॉडल ट्रेन करें

जैसा कि आपने पहले ही समझ लिया होगा, *price* कॉलम `परिणाम` वेरिएबल है जबकि *package* कॉलम `पूर्वानुमानक` वेरिएबल है।

इसके लिए, हम पहले डेटा को इस तरह विभाजित करेंगे कि 80% प्रशिक्षण सेट में जाए और 20% टेस्ट सेट में। फिर एक रेसिपी परिभाषित करेंगे जो पूर्वानुमानक कॉलम को एक सेट ऑफ इंटीजर में एन्कोड करेगी, और फिर एक मॉडल स्पेसिफिकेशन बनाएंगे। हम अपनी रेसिपी को प्रेप और बेक नहीं करेंगे क्योंकि हमें पहले से ही पता है कि यह डेटा को अपेक्षित तरीके से प्री-प्रोसेस करेगी।


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

शानदार काम! अब जब हमारे पास एक रेसिपी और मॉडल स्पेसिफिकेशन है, तो हमें इन्हें एक ऑब्जेक्ट में जोड़ने का तरीका ढूंढना होगा, जो पहले डेटा को प्रीप्रोसेस करेगा (पर्दे के पीछे prep+bake), प्रीप्रोसेस किए गए डेटा पर मॉडल को फिट करेगा और संभावित पोस्ट-प्रोसेसिंग गतिविधियों की भी अनुमति देगा। यह आपके मन की शांति के लिए कैसा है!🤩

Tidymodels में, इस सुविधाजनक ऑब्जेक्ट को [`workflow`](https://workflows.tidymodels.org/) कहा जाता है और यह आपके मॉडलिंग घटकों को आसानी से संग्रहीत करता है! इसे हम *Python* में *pipelines* कहेंगे।

तो चलिए सब कुछ एक workflow में जोड़ते हैं!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

साथ ही, एक वर्कफ़्लो को लगभग उसी तरह फिट/ट्रेन किया जा सकता है जैसे एक मॉडल को किया जाता है।


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

मॉडल प्रशिक्षण के दौरान सीखे गए गुणांक आउटपुट में देखे जा सकते हैं। ये गुणांक उस सर्वश्रेष्ठ फिट लाइन के गुणांक को दर्शाते हैं, जो वास्तविक और अनुमानित चर के बीच समग्र त्रुटि को न्यूनतम करता है।

#### टेस्ट सेट का उपयोग करके मॉडल के प्रदर्शन का मूल्यांकन करें

अब समय आ गया है यह देखने का कि मॉडल ने कैसा प्रदर्शन किया 📏! हम यह कैसे करेंगे?

अब जब हमने मॉडल को प्रशिक्षित कर लिया है, तो हम इसका उपयोग `parsnip::predict()` के माध्यम से टेस्ट सेट के लिए भविष्यवाणियां करने में कर सकते हैं। फिर हम इन भविष्यवाणियों की तुलना वास्तविक लेबल मानों से करेंगे ताकि यह मूल्यांकन कर सकें कि मॉडल कितना अच्छा (या नहीं!) काम कर रहा है।

आइए टेस्ट सेट के लिए भविष्यवाणियां करने से शुरू करें और फिर इन कॉलम्स को टेस्ट सेट के साथ जोड़ें।


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

हाँ, आपने अभी एक मॉडल को प्रशिक्षित किया और इसका उपयोग भविष्यवाणियाँ करने के लिए किया!🔮 क्या यह अच्छा है? चलिए मॉडल के प्रदर्शन का मूल्यांकन करते हैं!

Tidymodels में, हम इसे `yardstick::metrics()` का उपयोग करके करते हैं! लीनियर रिग्रेशन के लिए, आइए निम्नलिखित मेट्रिक्स पर ध्यान दें:

-   `रूट मीन स्क्वायर एरर (RMSE)`: [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) का वर्गमूल। यह एक पूर्ण मेट्रिक देता है जो लेबल (इस मामले में, कद्दू की कीमत) के समान इकाई में होता है। इसका मान जितना छोटा होगा, मॉडल उतना बेहतर होगा (सरल शब्दों में, यह औसत कीमत को दर्शाता है जिससे भविष्यवाणियाँ गलत होती हैं!)

-   `निर्धारण गुणांक (आमतौर पर R-squared या R2 के रूप में जाना जाता है)`: एक सापेक्ष मेट्रिक जिसमें मान जितना अधिक होगा, मॉडल का फिट उतना बेहतर होगा। मूल रूप से, यह मेट्रिक दर्शाता है कि मॉडल भविष्यवाणी और वास्तविक लेबल मानों के बीच के अंतर को कितना समझा सकता है।


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

मॉडल का प्रदर्शन गिर गया। चलिए देखते हैं कि क्या हम पैकेज और कीमत का स्कैटर प्लॉट बनाकर और फिर भविष्यवाणियों का उपयोग करके एक सर्वश्रेष्ठ फिट लाइन को ओवरले करके बेहतर संकेत प्राप्त कर सकते हैं।

इसका मतलब है कि हमें टेस्ट सेट को तैयार और प्रोसेस करना होगा ताकि पैकेज कॉलम को एन्कोड किया जा सके और फिर इसे हमारे मॉडल द्वारा की गई भविष्यवाणियों के साथ जोड़ना होगा।


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


जैसा कि आप देख सकते हैं, लीनियर रिग्रेशन मॉडल वास्तव में पैकेज और उसकी संबंधित कीमत के बीच के संबंध को अच्छी तरह से सामान्यीकृत नहीं करता है।

🎃 बधाई हो, आपने अभी-अभी एक ऐसा मॉडल बनाया है जो कुछ प्रकार की कद्दू की कीमत का अनुमान लगाने में मदद कर सकता है। आपका छुट्टी का कद्दू का खेत सुंदर होगा। लेकिन आप शायद एक बेहतर मॉडल बना सकते हैं!

## 5. एक पॉलिनोमियल रिग्रेशन मॉडल बनाएं

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>डसानी मदीपल्ली द्वारा इन्फोग्राफिक</figcaption>


<!--![डसानी मदीपल्ली द्वारा इन्फोग्राफिक](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.hi.png){width="800"}-->


कभी-कभी हमारे डेटा में रैखिक संबंध नहीं होता है, लेकिन फिर भी हम परिणाम की भविष्यवाणी करना चाहते हैं। बहुपद प्रतिगमन (Polynomial Regression) हमें अधिक जटिल गैर-रैखिक संबंधों के लिए भविष्यवाणी करने में मदद कर सकता है।

उदाहरण के लिए, हमारे कद्दू डेटा सेट में पैकेज और कीमत के बीच संबंध को लें। जबकि कभी-कभी चर के बीच रैखिक संबंध होता है - जैसे कि कद्दू का आकार बड़ा होने पर कीमत अधिक होती है - लेकिन कभी-कभी इन संबंधों को एक समतल या सीधी रेखा के रूप में नहीं दर्शाया जा सकता।

> ✅ [यहां कुछ और उदाहरण](https://online.stat.psu.edu/stat501/lesson/9/9.8) दिए गए हैं, जहां बहुपद प्रतिगमन का उपयोग किया जा सकता है।
>
> पिछले प्लॉट में Variety और Price के बीच संबंध को फिर से देखें। क्या यह scatterplot ऐसा लगता है कि इसे सीधी रेखा से विश्लेषित किया जाना चाहिए? शायद नहीं। इस मामले में, आप बहुपद प्रतिगमन का प्रयास कर सकते हैं।
>
> ✅ बहुपद गणितीय अभिव्यक्तियां हैं, जो एक या अधिक चर और गुणांक से बनी हो सकती हैं।

#### प्रशिक्षण सेट का उपयोग करके बहुपद प्रतिगमन मॉडल तैयार करें

बहुपद प्रतिगमन एक *घुमावदार रेखा* बनाता है ताकि गैर-रैखिक डेटा को बेहतर तरीके से फिट किया जा सके।

आइए देखें कि क्या बहुपद मॉडल भविष्यवाणी करने में बेहतर प्रदर्शन करेगा। हम पहले की तरह एक समान प्रक्रिया का पालन करेंगे:

-   एक रेसिपी बनाएं जो यह निर्दिष्ट करे कि हमारे डेटा को मॉडलिंग के लिए तैयार करने के लिए कौन-कौन से पूर्व-प्रसंस्करण चरण किए जाने चाहिए, जैसे: भविष्यवक्ताओं को एन्कोड करना और डिग्री *n* के बहुपदों की गणना करना।

-   एक मॉडल विनिर्देशन तैयार करें।

-   रेसिपी और मॉडल विनिर्देशन को एक वर्कफ़्लो में बांधें।

-   वर्कफ़्लो को फिट करके एक मॉडल बनाएं।

-   परीक्षण डेटा पर मॉडल के प्रदर्शन का मूल्यांकन करें।

चलो शुरू करते हैं!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### मॉडल के प्रदर्शन का मूल्यांकन करें

👏👏 आपने एक बहुपद मॉडल बनाया है, अब टेस्ट सेट पर भविष्यवाणियां करें!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

वाह-वाह, चलो देखें कि मॉडल ने परीक्षण_सेट पर `yardstick::metrics()` का उपयोग करके कैसा प्रदर्शन किया।


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 प्रदर्शन काफी बेहतर हुआ।

`rmse` लगभग 7 से घटकर लगभग 3 हो गया है, जो यह संकेत देता है कि वास्तविक कीमत और अनुमानित कीमत के बीच त्रुटि कम हुई है। आप इसे *आम तौर पर* इस तरह समझ सकते हैं कि औसतन, गलत भविष्यवाणियां लगभग \$3 तक गलत होती हैं। `rsq` लगभग 0.4 से बढ़कर 0.8 हो गया है।

ये सभी मेट्रिक्स दिखाते हैं कि बहुपद मॉडल रैखिक मॉडल की तुलना में कहीं बेहतर प्रदर्शन करता है। शानदार काम!

चलो इसे विज़ुअलाइज़ करने की कोशिश करते हैं!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


आप अपने डेटा के लिए एक घुमावदार रेखा देख सकते हैं जो इसे बेहतर तरीके से फिट करती है! 🤩

आप इसे और अधिक स्मूथ बना सकते हैं `geom_smooth` में एक बहुपद सूत्र पास करके, इस तरह:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

एक चिकनी वक्र की तरह!🤩

यहां बताया गया है कि आप एक नई भविष्यवाणी कैसे करेंगे:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


`polynomial model` की भविष्यवाणी सही लगती है, खासकर जब आप `price` और `package` के scatter plots को देखते हैं! और अगर यह मॉडल पिछले मॉडल से बेहतर है, तो वही डेटा देखते हुए, आपको इन महंगे कद्दुओं के लिए बजट बनाना होगा!

🏆 बहुत बढ़िया! आपने एक ही पाठ में दो regression मॉडल बनाए। Regression के अंतिम भाग में, आप logistic regression के बारे में सीखेंगे, जो श्रेणियों को निर्धारित करने में मदद करता है।

## **🚀चुनौती**

इस नोटबुक में कई अलग-अलग variables का परीक्षण करें और देखें कि correlation मॉडल की सटीकता से कैसे संबंधित है।

## [**पाठ के बाद का क्विज़**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **समीक्षा और स्व-अध्ययन**

इस पाठ में हमने Linear Regression के बारे में सीखा। Regression के अन्य महत्वपूर्ण प्रकार भी हैं। Stepwise, Ridge, Lasso और Elasticnet तकनीकों के बारे में पढ़ें। इस विषय पर अधिक जानने के लिए [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning) एक अच्छा कोर्स है।

यदि आप अद्भुत Tidymodels framework का उपयोग करना सीखना चाहते हैं, तो निम्नलिखित संसाधनों को देखें:

-   Tidymodels वेबसाइट: [Tidymodels के साथ शुरुआत करें](https://www.tidymodels.org/start/)

-   मैक्स कुह्न और जूलिया सिल्ज, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **धन्यवाद:**

[एलिसन हॉर्स्ट](https://twitter.com/allison_horst?lang=en) को, जिन्होंने अद्भुत चित्रण बनाए, जो R को अधिक स्वागतयोग्य और आकर्षक बनाते हैं। उनके और चित्रण [गैलरी](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM) में देखें।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
